<a href="https://colab.research.google.com/github/limshaocong/SysBERT/blob/main/t3_finetuning_seqlabel_cr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preliminaries**

In [ ]:
! pip install --user datasets transformers torch

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-15f9ec18-daf1-a300-6ea5-a81d4b7c0579)


# **Import & Pre-process Data**

In [3]:
model_type_dict = {
    'bert-base-cased' : 'bert-base-cased',
    'roberta-base' : 'roberta-base',
    'allenai/scibert_scivocab_cased' : 'allenai/scibert_scivocab_cased',
    'limsc/reqbert-tapt-epoch29' : 'bert-base-cased', # preferred
    'limsc/reqbert-tapt-epoch30' : 'bert-base-cased',
    'limsc/reqroberta-tapt-epoch20' : 'roberta-base',
    'limsc/reqroberta-tapt-epoch33' : 'roberta-base',
    'limsc/reqroberta-tapt-epoch43' : 'roberta-base', # preferred
    'limsc/reqroberta-tapt-epoch50' : 'roberta-base',
    'limsc/reqscibert-tapt-epoch10' : 'allenai/scibert_scivocab_cased', # preferred
    'limsc/reqscibert-tapt-epoch20' : 'allenai/scibert_scivocab_cased', # preferred
    'limsc/reqscibert-tapt-epoch31' : 'allenai/scibert_scivocab_cased',
    'limsc/reqscibert-tapt-epoch49' : 'allenai/scibert_scivocab_cased',
}

model_name_dict = {
    'bert-base-cased' : 'bert',
    'roberta-base' : 'roberta',
    'allenai/scibert_scivocab_cased' : 'scibert',
    'limsc/reqbert-tapt-epoch29' : 'reqbert-e29',
    'limsc/reqbert-tapt-epoch30' : 'reqbert-e30',
    'limsc/reqroberta-tapt-epoch20' : 'reqroberta-e20',
    'limsc/reqroberta-tapt-epoch33' : 'reqroberta-e33',
    'limsc/reqroberta-tapt-epoch43' : 'reqroberta-e43',
    'limsc/reqroberta-tapt-epoch50' : 'reqroberta-e50',
    'limsc/reqscibert-tapt-epoch10' : 'reqscibert-e10',
    'limsc/reqscibert-tapt-epoch20' : 'reqscibert-e20',
    'limsc/reqscibert-tapt-epoch31' : 'reqscibert-e31',
    'limsc/reqscibert-tapt-epoch49' : 'reqscibert-e49',
}

task_name_dict = {
    'limsc/fr-nfr-classification' : 'frnfr',
    'limsc/nfr-subclass-classification' : 'subclass',
    'limsc/concept-recognition' : 'cr',
    'limsc/concept-recognition-not-iob' : 'cr',
    'limsc/sysmlv2-entity-extraction' : 'ee'
}

In [4]:
from datasets import load_dataset

ds_name = 'limsc/concept-recognition-not-iob'
ds = load_dataset(ds_name)
ds

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Using custom data configuration limsc--concept-recognition-not-iob-49ad4a4453826183


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/limsc___parquet/limsc--concept-recognition-not-iob-49ad4a4453826183/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'tokens', 'concept_tags'],
        num_rows: 132
    })
    val: Dataset({
        features: ['id', 'tokens', 'concept_tags'],
        num_rows: 131
    })
    train: Dataset({
        features: ['id', 'tokens', 'concept_tags'],
        num_rows: 611
    })
})

In [5]:
label_list = ds["train"].features['concept_tags'].feature.names
label_list

['Materials / EEEs',
 'O',
 'GN&C',
 'Thermal',
 'Parameter',
 'Quality control',
 'Safety / Risk (Control)',
 'System engineering',
 'Space Environment',
 'Cleanliness',
 'Measurement',
 'Telecom.',
 'Project Organisation / Documentation',
 'Project Scope',
 'Power',
 'OBDH',
 'Structure & Mechanism',
 'Nonconformity',
 'Propulsion']

In [6]:
model_checkpoint = 'bert-base-cased'

In [7]:
from transformers import AutoTokenizer

prefix_space = True if model_type_dict[model_checkpoint] == 'roberta-base' else False

tokenizer = AutoTokenizer.from_pretrained(
    model_type_dict[model_checkpoint],
    use_fast = True,
    add_prefix_space = prefix_space
)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
def tokenize_and_align_labels(examples, label_all_tokens = False):

    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation = True,
        is_split_into_words = True
    )
    labels = []
    
    for i, label in enumerate(examples['concept_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:           
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.

            if word_idx is None:
                label_ids.append(-100)
            
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [9]:
tokenized_ds = ds.map(tokenize_and_align_labels, batched = True)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f3b5677c680> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# **Model Fine-tuning (Single Loop)**

In [10]:
from transformers import DataCollatorForTokenClassification

batch_size = 16

data_collator = DataCollatorForTokenClassification(
    tokenizer = tokenizer,
    padding = True,
    return_tensors = 'tf'
)

def batching(tokenized_ds, batch_size):

    batched_train_ds = tokenized_ds['train'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = True,
        batch_size = batch_size,
        collate_fn = data_collator,
    )

    batched_val_ds = tokenized_ds['val'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = False,
        batch_size = batch_size,
        collate_fn = data_collator,
    )

    batched_test_ds = tokenized_ds['test'].to_tf_dataset(
        columns = ['attention_mask', 'input_ids', 'labels'],
        shuffle = False,
        batch_size = batch_size,
        collate_fn = data_collator,
    )
    
    return batched_train_ds, batched_val_ds, batched_test_ds

batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

/root/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [14]:
import tensorflow as tf
from transformers import TFAutoModelForTokenClassification, create_optimizer

seed = 6789767
tf.random.set_seed(seed)

num_epochs = 1
initial_lr = 2e-5

def create_model(num_epochs, initial_lr):

    model = TFAutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels = len(label_list)
    )

    batches_per_epoch = len(tokenized_ds['train']) // batch_size
    total_train_steps = int(batches_per_epoch * num_epochs)

    optimizer, schedule = create_optimizer(
        init_lr = initial_lr,
        num_warmup_steps = total_train_steps // 20,
        num_train_steps = total_train_steps,
        weight_decay_rate = 0.01
    )

    model.compile(optimizer = optimizer)

    return model

model = create_model(num_epochs, initial_lr)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [12]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, CSVLogger

class update_logger(Callback):

    def __init__(self):    
        super(update_logger, self).__init__()

    def on_epoch_end(self, epoch, logs = {}):
        logs['seed'] = seed
        logs['batch_size'] = batch_size
        logs['learning_rate'] = initial_lr

update_logger_cb = update_logger()

csvlogger_file = f'{model_name_dict[model_checkpoint]}-{task_name_dict[ds_name]}.csv'
csvlogger_cb = CSVLogger(csvlogger_file, append = True)

callbacks = [update_logger_cb, csvlogger_cb]

In [15]:
model.fit(
    batched_train_ds,
    validation_data = batched_val_ds,
    epochs = num_epochs,
    callbacks = callbacks
)

/root/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


39/39 [==============================] - 43s 606ms/step - loss: 1.5821 - val_loss: 1.2162 - seed: 6789767.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05


# **Hyperparameter tuning**

In [ ]:
batch_sizes = [8] if model_type_dict[model_checkpoint] == 'roberta-base' else [8, 16]
initial_lrs = [5e-5, 3e-5, 2e-5]
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

num_epochs = 5

for batch_size in batch_sizes:

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)
    
    for initial_lr in initial_lrs:
    
        for seed in seeds:
    
            tf.random.set_seed(seed)
            model = create_model(num_epochs, initial_lr)

            csvlogger_file = f'{model_name_dict[model_checkpoint]}-{task_name_dict[ds_name]}.csv'
            csvlogger_cb = CSVLogger(csvlogger_file, append = True)

            callbacks = [update_logger_cb, csvlogger_cb]

            model.fit(
                batched_train_ds,
                validation_data = batched_val_ds,
                epochs = num_epochs,
                callbacks = callbacks
            )

/root/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you 

Epoch 1/5
76/76 [==============================] - 30s 201ms/step - loss: 1.3934 - val_loss: 0.8224 - seed: 21916.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 2/5
76/76 [==============================] - 13s 168ms/step - loss: 0.6133 - val_loss: 0.5671 - seed: 21916.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 3/5
76/76 [==============================] - 13s 166ms/step - loss: 0.3711 - val_loss: 0.5052 - seed: 21916.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 4/5
76/76 [==============================] - 13s 163ms/step - loss: 0.2369 - val_loss: 0.5099 - seed: 21916.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 5/5
76/76 [==============================] - 13s 167ms/step - loss: 0.1743 - val_loss: 0.5249 - seed: 21916.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/5
76/76 [==============================] - 30s 201ms/step - loss: 1.3973 - val_loss: 0.9366 - seed: 25412.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 2/5
76/76 [==============================] - 12s 163ms/step - loss: 0.6913 - val_loss: 0.6081 - seed: 25412.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 3/5
76/76 [==============================] - 12s 163ms/step - loss: 0.4025 - val_loss: 0.5586 - seed: 25412.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 4/5
76/76 [==============================] - 13s 165ms/step - loss: 0.2572 - val_loss: 0.5455 - seed: 25412.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05
Epoch 5/5
76/76 [==============================] - 13s 166ms/step - loss: 0.1870 - val_loss: 0.5467 - seed: 25412.0000 - batch_size: 8.0000 - learning_rate: 5.0000e-05


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


...

Epoch 1/5
38/38 [==============================] - 31s 448ms/step - loss: 1.7338 - val_loss: 1.1478 - seed: 86349.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 2/5
38/38 [==============================] - 13s 358ms/step - loss: 1.0754 - val_loss: 1.0255 - seed: 86349.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 3/5
38/38 [==============================] - 13s 347ms/step - loss: 0.9006 - val_loss: 0.8863 - seed: 86349.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 4/5
38/38 [==============================] - 14s 363ms/step - loss: 0.7675 - val_loss: 0.8129 - seed: 86349.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 5/5
38/38 [==============================] - 13s 337ms/step - loss: 0.7015 - val_loss: 0.7848 - seed: 86349.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/5
38/38 [==============================] - 31s 457ms/step - loss: 1.8019 - val_loss: 1.1431 - seed: 12082.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 2/5
38/38 [==============================] - 13s 353ms/step - loss: 1.0693 - val_loss: 0.9854 - seed: 12082.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 3/5
38/38 [==============================] - 14s 362ms/step - loss: 0.8654 - val_loss: 0.8441 - seed: 12082.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 4/5
38/38 [==============================] - 14s 362ms/step - loss: 0.7307 - val_loss: 0.7663 - seed: 12082.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 5/5
38/38 [==============================] - 13s 342ms/step - loss: 0.6643 - val_loss: 0.7453 - seed: 12082.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/5
38/38 [==============================] - 31s 417ms/step - loss: 1.8246 - val_loss: 1.1301 - seed: 81996.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 2/5
38/38 [==============================] - 13s 355ms/step - loss: 1.0491 - val_loss: 0.9758 - seed: 81996.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 3/5
38/38 [==============================] - 13s 352ms/step - loss: 0.8521 - val_loss: 0.8437 - seed: 81996.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 4/5
38/38 [==============================] - 14s 355ms/step - loss: 0.7317 - val_loss: 0.7784 - seed: 81996.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05
Epoch 5/5
38/38 [==============================] - 14s 355ms/step - loss: 0.6668 - val_loss: 0.7638 - seed: 81996.0000 - batch_size: 16.0000 - learning_rate: 2.0000e-05


# **Model Evaluation**

In [24]:
mod_hyperparam_dict = {
    'bert-base-cased' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2},
    'roberta-base' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'allenai/scibert_scivocab_cased' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2},
    'limsc/reqbert-tapt-epoch29' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2},
    'limsc/reqroberta-tapt-epoch43' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'limsc/reqscibert-tapt-epoch20' : {'batch_size' : 8, 'initial_lr' : 5e-5, 'num_epochs' : 2}
}

In [27]:
import numpy as np
from sklearn.metrics import classification_report

mods = list(mod_hyperparam_dict.keys())
results_list = []
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]
seeds = [seeds[0]]

for mod in mods:

    model_checkpoint = mod

    batch_size = mod_hyperparam_dict[mod]['batch_size']
    initial_lr = mod_hyperparam_dict[mod]['initial_lr']
    num_epochs = mod_hyperparam_dict[mod]['num_epochs']

    tokenizer = AutoTokenizer.from_pretrained(
        model_type_dict[mod],
        use_fast = True
    )

    tokenized_ds = ds.map(tokenize_and_align_labels, batched = True)
    
    data_collator = DataCollatorForTokenClassification(
        tokenizer = tokenizer,
        padding = True,
        return_tensors = 'tf'
    )

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

    for seed in seeds:

        tf.random.set_seed(seed)
        model = create_model(num_epochs, initial_lr)

        model.fit(
            batched_train_ds,
            epochs = num_epochs
        )

        all_preds = []
        all_trues = []

        for batch in batched_test_ds:

            y_preds_logits = model.predict(batch)['logits']
            y_preds = np.argmax(y_preds_logits, axis = 2)
            y_trues = batch['labels']
            p = y_preds, y_trues

            all_pred = [
                [label_list[p] for (p, l) in zip(y_pred, y_true) if l != -100]
                for y_pred, y_true in zip(y_preds, y_trues)]
            all_true = [
                [label_list[l] for (p, l) in zip(y_pred, y_true) if l != -100]
                for y_pred, y_true in zip(y_preds, y_trues)]
            
            all_preds.extend([item for sublist in all_pred for item in sublist])
            all_trues.extend([item for sublist in all_true for item in sublist])
            
            rep = classification_report(
                all_trues,
                all_preds,
                output_dict = True
            )

            result_dict = {
                    'model': mod,
                    'seed' : seed,
                    'CLEAN_p' : format(rep['Cleanliness']['precision'], '.4f'),
                    'CLEAN_r' : format(rep['Cleanliness']['recall'], '.4f'),
                    'CLEAN_f1' : format(rep['Cleanliness']['f1-score'], '.4f'),
                    'GNC_p' : format(rep['GN&C']['precision'], '.4f'),
                    'GNC_r' : format(rep['GN&C']['recall'], '.4f'),
                    'GNC_f1' : format(rep['GN&C']['f1-score'], '.4f'),
                    'MEEE_p' : format(rep['Materials / EEEs']['precision'], '.4f'),
                    'MEEE_r' : format(rep['Materials / EEEs']['recall'], '.4f'),
                    'MEEE_f1' : format(rep['Materials / EEEs']['f1-score'], '.4f'),
                    'MEA_p' : format(rep['Measurement']['precision'], '.4f'),
                    'MEA_r' : format(rep['Measurement']['recall'], '.4f'),
                    'MEA_f1' : format(rep['Measurement']['f1-score'], '.4f'),
                    'NC_p' : format(rep['Nonconformity']['precision'], '.4f'),
                    'NC_r' : format(rep['Nonconformity']['recall'], '.4f'),
                    'NC_f1' : format(rep['Nonconformity']['f1-score'], '.4f'),
                    'OBDH_p' : format(rep['OBDH']['precision'], '.4f'),
                    'OBDH_r' : format(rep['OBDH']['recall'], '.4f'),
                    'OBDH_f1' : format(rep['OBDH']['f1-score'], '.4f'),
                    'PARAM_p' : format(rep['Parameter']['precision'], '.4f'),
                    'PARAM_r' : format(rep['Parameter']['recall'], '.4f'),
                    'PARAM_f1' : format(rep['Parameter']['f1-score'], '.4f'),
                    'POWER_p' : format(rep['Power']['precision'], '.4f'),
                    'POWER_r' : format(rep['Power']['recall'], '.4f'),
                    'POWER_f1' : format(rep['Power']['f1-score'], '.4f'),
                    'POD_p' : format(rep['Project Organisation / Documentation']['precision'], '.4f'),
                    'POD_r' : format(rep['Project Organisation / Documentation']['recall'], '.4f'),
                    'POD_f1' : format(rep['Project Organisation / Documentation']['f1-score'], '.4f'),
                    'PS_p' : format(rep['Project Scope']['precision'], '.4f'),
                    'PS_r' : format(rep['Project Scope']['recall'], '.4f'),
                    'PS_f1' : format(rep['Project Scope']['f1-score'], '.4f'),
                    'PROP_p' : format(rep['Propulsion']['precision'], '.4f'),
                    'PROP_r' : format(rep['Propulsion']['recall'], '.4f'),
                    'PROP_f1' : format(rep['Propulsion']['f1-score'], '.4f'),
                    'QC_p' : format(rep['Quality control']['precision'], '.4f'),
                    'QC_r' : format(rep['Quality control']['recall'], '.4f'),
                    'QC_f1' : format(rep['Quality control']['f1-score'], '.4f'),
                    'SRC_p' : format(rep['Safety / Risk (Control)']['precision'], '.4f'),
                    'SRC_r' : format(rep['Safety / Risk (Control)']['recall'], '.4f'),
                    'SRC_f1' : format(rep['Safety / Risk (Control)']['f1-score'], '.4f'),
                    'SPACEE_p' : format(rep['Space Environment']['precision'], '.4f'),
                    'SPACEE_r' : format(rep['Space Environment']['recall'], '.4f'),
                    'SPACEE_f1' : format(rep['Space Environment']['f1-score'], '.4f'),
                    'SM_p' : format(rep['Structure & Mechanism']['precision'], '.4f'),
                    'SM_r' : format(rep['Structure & Mechanism']['recall'], '.4f'),
                    'SM_f1' : format(rep['Structure & Mechanism']['f1-score'], '.4f'),
                    'SYSE_p' : format(rep['System engineering']['precision'], '.4f'),
                    'SYSE_r' : format(rep['System engineering']['recall'], '.4f'),
                    'SYSE_f1' : format(rep['System engineering']['f1-score'], '.4f'),
                    'TELE_p' : format(rep['Telecom.']['precision'], '.4f'),
                    'TELE_r' : format(rep['Telecom.']['recall'], '.4f'),
                    'TELE_f1' : format(rep['Telecom.']['f1-score'], '.4f'),
                    'THERM_p' : format(rep['Thermal']['precision'], '.4f'),
                    'THERM_r' : format(rep['Thermal']['recall'], '.4f'),
                    'THERM_f1' : format(rep['Thermal']['f1-score'], '.4f'),
                    'macro_p' : format(rep['macro avg']['precision'], '.4f'),
                    'macro_r' : format(rep['macro avg']['recall'], '.4f'),
                    'macro_f1' : format(rep['macro avg']['f1-score'], '.4f'),
                    'weight_p' : format(rep['weighted avg']['precision'], '.4f'),
                    'weight_r' : format(rep['weighted avg']['recall'], '.4f'),
                    'weight_f1' : format(rep['weighted avg']['f1-score'], '.4f')
                }

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/root/.local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you 

Epoch 1/2
77/77 [==============================] - 32s 214ms/step - loss: 1.2370
Epoch 2/2
77/77 [==============================] - 17s 219ms/step - loss: 0.6288


ValueError: ignored

In [34]:
print(classification_report(all_trues, all_preds))

                                      precision    recall  f1-score   support

                    Materials / EEEs       0.12      1.00      0.22         1
                         Measurement       1.00      1.00      1.00         6
                       Nonconformity       0.00      0.00      0.00         1
                                   O       0.94      0.96      0.95       109
                                OBDH       0.00      0.00      0.00         0
                           Parameter       0.00      0.00      0.00         3
                               Power       0.00      0.00      0.00         3
Project Organisation / Documentation       0.00      0.00      0.00         1
                       Project Scope       1.00      0.25      0.40         4
                          Propulsion       0.50      0.17      0.25         6
                     Quality control       0.00      0.00      0.00         4
                   Space Environment       0.00      0.00      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [22]:
from sklearn.metrics import classification_report

print(classification_report(all_trues, all_preds))
cr = classification_report(all_trues, all_preds, output_dict = True)

cr['Cleanliness']['precision']

                                      precision    recall  f1-score   support

                         Cleanliness       0.00      0.00      0.00        28
                                GN&C       0.00      0.00      0.00        54
                    Materials / EEEs       0.00      0.00      0.00        79
                         Measurement       0.00      0.00      0.00        78
                       Nonconformity       0.00      0.00      0.00        23
                                   O       0.68      1.00      0.81      3028
                                OBDH       0.00      0.00      0.00       104
                           Parameter       0.00      0.00      0.00       103
                               Power       0.00      0.00      0.00       166
Project Organisation / Documentation       0.00      0.00      0.00        29
                       Project Scope       0.00      0.00      0.00        66
                          Propulsion       0.00      0.00      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

0.0

In [ ]:
import pandas as pd

pd.DataFrame(cr)

# **Prediction Pipeline**

In [ ]:
from transformers import TokenClassificationPipeline

pipe = TokenClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    # aggregation_strategy = 'simple'
)

In [ ]:
text = 'The Micro-VCM datasheet shall contain the product type.'
text = 'Storage conditions shall prevent the degredation of the structure.'
text = 'A record of the process data shall be part of the process procedure'
text = 'In case of doubt, the internal NRB shall classify nonconformances as major.'
text = 'The Reserved shall be an 8-bit field that is set to 0x00.'

pipe(text)